In [2]:
## --- Smart fix: detect true PROJECT_ROOT and solve LP (python-mip) ---
from pathlib import Path
from datetime import datetime
from time import perf_counter
import numpy as np

# 1) Detect if you're inside 'notebooks/' and step up
here = Path.cwd()
PROJECT_ROOT = here.parent if here.name == "notebooks" else here
print(f"[info] PROJECT_ROOT = {PROJECT_ROOT}")

# 2) Search for LP file intelligently
def find_lp() -> Path:
    results_lp = PROJECT_ROOT / "results" / "baseline.lp"
    known_lp = PROJECT_ROOT / "data" / "1" / "31bonds" / "docplex-bin-avgonly.lp"
    if results_lp.exists():
        print(f"[ok] Using LP: {results_lp}")
        return results_lp
    if known_lp.exists():
        print(f"[ok] Using known LP: {known_lp}")
        return known_lp
    matches = list(PROJECT_ROOT.rglob("*.lp"))
    if matches:
        preferred = [p for p in matches if any(part in ("data", "results") for part in p.parts)]
        chosen = preferred[0] if preferred else matches[0]
        print("[warn] No default LP found. Discovered these instead:")
        for i, p in enumerate(matches[:10], 1):
            print(f"  {i}. {p}")
        print(f"[use] Selected: {chosen}")
        return chosen
    raise FileNotFoundError(
        "No .lp file found anywhere under project.\n"
        "→ Drop one into /data/ or /results/, or export via model.write(...)."
    )

chosen_lp = find_lp()

# 🔍 LP file preview — safe and portable

lp_full_path = chosen_lp.resolve()

if lp_full_path.exists():
    with open(lp_full_path, "r") as f:
        lp_contents = f.read()
    print("✅ LP file loaded successfully")
    print(f"[file] {lp_full_path}")
    print(lp_contents[:500])  # Optional preview
else:
    print(f"❌ LP file not found at: {lp_full_path}")

# 3) Build relative path
try:
    lp_rel = str(chosen_lp.resolve().relative_to(PROJECT_ROOT.resolve()))
except ValueError:
    lp_rel = str(chosen_lp.name)
    print(f"[warn] LP path not relative to project root. Using name: {lp_rel}")

# 4) Define and run solver (python-mip)
def solve_mip_tuned(lp_rel, max_seconds=20.0, mip_gap=0.0):
    from mip import Model, OptimizationStatus, BINARY
    lp_path = (PROJECT_ROOT / lp_rel).resolve()
    m = Model()
    m.verbose = 1
    try: m.max_seconds = max_seconds
    except: pass
    try: m.max_mip_gap = mip_gap
    except: pass
    m.read(str(lp_path))
    t0 = perf_counter()
    status = m.optimize()
    dt = perf_counter() - t0
    x = [int(round(v.x)) for v in m.vars]
    fx = m.objective_value if status.name in ("OPTIMAL", "FEASIBLE") else None
    return x, fx, dt

# 5) Run solver and summarize
print(f"[run] Solving LP: {lp_rel}")
x2, fx2, dt2 = solve_mip_tuned(lp_rel)

summary = {
    "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "LP file": lp_rel,
    "Bonds selected": int(np.sum(x2)),
    "Objective": round(fx2, 5) if fx2 else None,
    "Runtime (s)": round(dt2, 3)
}

print("— Results —")
for k, v in summary.items():
    print(f"{k}: {v}")


[info] PROJECT_ROOT = C:\Users\Admin\Documents\WISER_Optimization_VG
[ok] Using known LP: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
[run] Solving LP: data\1\31bonds\docplex-bin-avgonly.lp
— Results —
Timestamp: 2025-08-11 03:02:44
LP file: data\1\31bonds\docplex-bin-avgonly.lp
Bonds selected: 18
Objective: 40.27711
Runtime (s): 43.581


In [1]:
import numpy as np


In [2]:
import sys
from pathlib import Path

# Detect project root (whether you're in notebooks/ or root)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"

# Add src/ to sys.path if not already there
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("✅ src added to path:", SRC_PATH.exists())


✅ src added to path: True


In [3]:
import sys
sys.path.append("..")  # Go up one level to access src folder

from src.experiment import solve_bond_selection
print("✅ Imported successfully")


▶ src.experiment imported and running
✅ Imported successfully


In [4]:
import os, sys, pathlib

# 1) Set working directory to your repo root (where README and src/ live)
repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

# 2) Ensure Python can import from src/
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG\notebooks


In [5]:
%cd C:\Users\Admin\Documents\WISER_Optimization_VG


C:\Users\Admin\Documents\WISER_Optimization_VG


C:\Users\Admin\Documents\WISER_Optimization_VG\.venv310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
import sys
from pathlib import Path

# Detect project root (whether you're in notebooks/ or root)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"

# Add src/ to sys.path if not already there
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("✅ src added to path:", SRC_PATH.exists())


✅ src added to path: True


In [7]:
sys.path.append(str(PROJECT_ROOT))
from src.experiment import solve_bond_selection




In [8]:
import os, sys, pathlib

repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG


In [9]:
import os, sys, pathlib

repo_root = pathlib.Path.cwd()
print("Working in:", repo_root)

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))


Working in: C:\Users\Admin\Documents\WISER_Optimization_VG


In [10]:
from pathlib import Path
import os

# Get current folder
cwd = Path.cwd()
print("Jupyter is running in:", cwd)

# Go up one level if you're inside /notebooks
if cwd.name == "notebooks":
    os.chdir(cwd.parent)
    print("Switched to:", Path.cwd())


Jupyter is running in: C:\Users\Admin\Documents\WISER_Optimization_VG


In [11]:
import sys
PROJECT_ROOT = Path.cwd()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("Root added to sys.path:", str(PROJECT_ROOT))


Root added to sys.path: C:\Users\Admin\Documents\WISER_Optimization_VG


In [12]:
def touch_init(path):
    path.mkdir(parents=True, exist_ok=True)
    init = path / "__init__.py"
    if not init.exists():
        init.touch()
        print("✅ Created __init__.py in", path)
    else:
        print("✔️ __init__.py already exists in", path)

# Apply to src/ and any nested package folders
for rel in ["src", "src/sbo", "src/sbo/src", "src/sbo/src/optimizer"]:
    folder = PROJECT_ROOT / rel
    if folder.exists():
        touch_init(folder)


✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src
✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo
✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo\src
✔️ __init__.py already exists in C:\Users\Admin\Documents\WISER_Optimization_VG\src\sbo\src\optimizer


In [13]:
exp_path = PROJECT_ROOT / "src" / "experiment.py"
if not exp_path.exists():
    raise FileNotFoundError("Missing: src/experiment.py")

text = exp_path.read_text(encoding="utf-8")
replacements = {
    "from .sbo.src.optimizer.optimization_monitor": "from src.sbo.src.optimizer.optimization_monitor",
    "from .sbo.src.optimizer.optimization_wrapper": "from src.sbo.src.optimizer.optimization_wrapper",
}

patched = text
for old, new in replacements.items():
    if old in patched:
        patched = patched.replace(old, new)

if patched != text:
    backup = exp_path.with_suffix(".py.bak")
    backup.write_text(text, encoding="utf-8")
    exp_path.write_text(patched, encoding="utf-8")
    print("✅ Patched experiment.py (backup saved as experiment.py.bak)")
else:
    print("✔️ No changes needed — looks clean.")


✔️ No changes needed — looks clean.


In [14]:
from src.experiment import solve_bond_selection
print("✅ Import OK")


✅ Import OK


In [15]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"  # you used this file earlier


In [16]:
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"
x, fx, dt = solve_bond_selection(LP_REL)
print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 40.4


In [17]:
import os
print(os.getcwd())


C:\Users\Admin\Documents\WISER_Optimization_VG


In [18]:
import sys
sys.path.append("..")  # Moves up one level to access src/

from src.experiment import solve_bond_selection
print("✅ Imported successfully")


✅ Imported successfully


In [19]:
import numpy as np

LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"  # Update if needed

x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 39.63


In [20]:
import os

for root, dirs, files in os.walk(".."):
    for name in files:
        if name.endswith(".lp"):
            print(os.path.join(root, name))


..\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly-nocplexvars.lp
..\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
..\WISER_Optimization_VG\data\1\31bonds\docplex-bin-minmaxavg.lp
..\WISER_Optimization_VG\data\1\31bonds\docplex.lp
..\WISER_Optimization_VG\data\1\full\docplex-full-reduced-nosemi.lp
..\WISER_Optimization_VG\data\1\full\docplex-orig.lp
..\WISER_Optimization_VG\data\1\gurobi\gurobi.lp


In [21]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"



In [22]:
x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 39.81


In [23]:
LP_PATH = "data/1/31bonds/docplex-bin-avgonly.lp"


In [24]:
x, fx, dt = solve_bond_selection(LP_PATH)

print("🔢 Bonds selected:", int(np.sum(x)))
print("📈 Objective value:", round(fx, 4))
print("⏱ Runtime (seconds):", round(dt, 2))


🔢 Bonds selected: 21
📈 Objective value: 40.2771
⏱ Runtime (seconds): 39.58


In [2]:
# Cell 25 — self-contained solve + results (fixes NameError)

# Imports
from pathlib import Path
from time import perf_counter
import numpy as np
import pandas as pd

# 1) Detect project root (works if running from repo root or notebooks/)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
print(f"[info] PROJECT_ROOT = {PROJECT_ROOT}")

# 2) Locate LP file
results_lp = PROJECT_ROOT / "results" / "baseline.lp"
known_lp = PROJECT_ROOT / "data" / "1" / "31bonds" / "docplex-bin-avgonly.lp"

def choose_lp() -> Path:
    if results_lp.exists():
        print(f"[ok] Using: {results_lp}")
        return results_lp
    if known_lp.exists():
        print(f"[ok] Using known LP: {known_lp}")
        return known_lp
    matches = list(PROJECT_ROOT.rglob("*.lp"))
    if matches:
        preferred = [p for p in matches if any(part in ("data", "results") for part in p.parts)]
        chosen = preferred[0] if preferred else matches[0]
        print("[warn] Falling back to discovered LP:")
        for i, p in enumerate(matches[:10], 1):
            print(f"  {i}. {p}")
        print(f"[use] Selected: {chosen}")
        return chosen
    raise FileNotFoundError("No .lp file found under the project. Place one in data/ or results/ and re-run.")

chosen = choose_lp()

# 3) Relative path (for reproducibility)
try:
    lp_rel = str(chosen.resolve().relative_to(PROJECT_ROOT.resolve()))
except ValueError:
    lp_rel = chosen.name
    print(f"[warn] LP is outside PROJECT_ROOT. Using name only: {lp_rel}")
print(f"[run] Solving relative LP: {lp_rel}")

# 4) Define the solver function (python-mip)
def solve_mip_tuned(lp_rel_path: str, max_seconds: float = 20.0, mip_gap: float = 0.0):
    from mip import Model, OptimizationStatus, BINARY
    lp_path = (PROJECT_ROOT / lp_rel_path).resolve()
    if not lp_path.exists():
        raise FileNotFoundError(f"LP file not found at: {lp_path}")

    m = Model()
    m.verbose = 1
    # Set limits if supported
    try: m.max_seconds = float(max_seconds)
    except Exception: pass
    try: m.max_mip_gap = float(mip_gap)
    except Exception: pass

    m.read(str(lp_path))
    t0 = perf_counter()
    status = m.optimize()
    dt = perf_counter() - t0

    fx = m.objective_value if status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE) else None
    # Prefer binary vars if present; else include all
    try:
        xs_bin = [int(round(v.x)) for v in m.vars if getattr(v, "var_type", None) == BINARY]
        x = xs_bin if xs_bin else [int(round(v.x)) for v in m.vars]
    except Exception:
        x = [int(round(v.x)) for v in m.vars]

    return x, fx, dt

# 5) Run solve
x2, fx2, dt2 = solve_mip_tuned(lp_rel, max_seconds=20, mip_gap=0.0)

# 6) Summarize and expose variables for later cells
def safe_sum(v):
    try:
        return int(np.sum(v))
    except Exception:
        try:
            return int(sum(v))
        except Exception:
            return None

tuned_results = {
    "LP file": lp_rel,
    "Bonds selected": safe_sum(x2),
    "Objective value": float(fx2) if fx2 is not None else None,
    "Runtime (s)": round(float(dt2), 3) if dt2 is not None else None,
    "Solver": "python-mip",
    "Strategy": "tuned (20s, gap=0.0)",
}

print("— Results —")
print(f"  Bonds selected: {tuned_results['Bonds selected']}")
print(f"  Objective:      {tuned_results['Objective value']}")
print(f"  Runtime (s):    {tuned_results['Runtime (s)']}")

# 7) Optional: create a DataFrame so later cells can display it
results_df = pd.DataFrame([tuned_results])
results_df


[info] PROJECT_ROOT = C:\Users\Admin\Documents\WISER_Optimization_VG
[ok] Using known LP: C:\Users\Admin\Documents\WISER_Optimization_VG\data\1\31bonds\docplex-bin-avgonly.lp
[run] Solving relative LP: data\1\31bonds\docplex-bin-avgonly.lp
— Results —
  Bonds selected: 21
  Objective:      40.27711275896016
  Runtime (s):    57.579


,LP file,Bonds selected,Objective value,Runtime (s),Solver,Strategy
0,data\1\31bonds\docplex-bin-avgonly.lp,21,40.277113,57.579,python-mip,"tuned (20s, gap=0.0)"


In [ ]:
LP_REL = os.path.join(PROJECT_ROOT, "results", "baseline.lp")


In [ ]:
os.path.exists(LP_REL)


In [ ]:
# Step 1: Set LP file path (relative to project root)
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"

# Sanity check the path
print("LP exists?", (PROJECT_ROOT / LP_REL).exists(), "->", (PROJECT_ROOT / LP_REL))

# Step 2: Run the solver (20s time limit)
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)

# Step 3: Print results
print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))



In [ ]:
# Imports
import os, sys, time
from pathlib import Path
import numpy as np

# If python-mip isn't imported yet
from mip import Model

# Detect project root (works whether your notebook is in repo root or in notebooks/)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

print("PROJECT_ROOT ->", PROJECT_ROOT)

def solve_mip_tuned(lp_rel_path, *, max_seconds=20, mip_gap=0.0, threads=0):
    """
    lp_rel_path: path relative to project root, e.g., 'data/1/31bonds/docplex-bin-avgonly.lp'
    max_seconds: time limit for the MIP solve
    mip_gap: allowed relative optimality gap (not all solvers honor this via API)
    threads: number of threads (0 lets solver decide)
    """
    lp_path = (PROJECT_ROOT / lp_rel_path).resolve()
    if not lp_path.exists():
        raise FileNotFoundError(f"LP file not found at: {lp_path}")

    start = time.perf_counter()
    m = Model()
    m.read(str(lp_path))

    # Optional knobs (wrapped in try/except to avoid solver-specific attribute issues)
    try:
        if threads:
            m.threads = threads
    except Exception:
        pass

    # Use the built-in time limit on optimize
    m.optimize(max_seconds=max_seconds)

    # Collect solution
    x = np.array([v.x for v in m.vars])
    fx = float(m.objective_value)
    dt = time.perf_counter() - start
    return x, fx, dt

print("✅ solve_mip_tuned is defined")


In [ ]:
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"


In [ ]:
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)


In [ ]:
print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))


In [ ]:
{
    "LP file": LP_REL,
    "Bonds selected": ...,
    "Objective value": ...,
    "Runtime (s)": ...,
    "Solver": "python-mip",
    "Strategy": "tuned (20s limit)"
}


In [ ]:
tuned_results = {
    "LP file": LP_REL,
    "Bonds selected": int(np.sum(x2)),
    "Objective value": round(fx2, 4),
    "Runtime (s)": round(dt2, 2),
    "Solver": "python-mip",
    "Strategy": "tuned (20s limit)"
}


In [2]:
# Import libraries
import numpy as np
import pandas as pd

# Define input values (make sure these match your earlier code or data)
x2 = np.array([1, 0, 1])            # Example bond selections
fx2 = 123.456                       # Example objective value
dt2 = 19.87                         # Example runtime in seconds
LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"  # Path to your LP file

# Define results dictionary
tuned_results = {
    "LP file": LP_REL,
    "Bonds selected": int(np.sum(x2)),
    "Objective value": round(fx2, 4),
    "Runtime (s)": round(dt2, 2),
    "Solver": "python-mip",
    "Strategy": "tuned (20s limit)"
}

# Convert to DataFrame
results_df = pd.DataFrame([tuned_results])

# Display the DataFrame
results_df



,LP file,Bonds selected,Objective value,Runtime (s),Solver,Strategy
0,data/1/31bonds/docplex-bin-avgonly.lp,2,123.456,19.87,python-mip,tuned (20s limit)


In [ ]:
import numpy as np
import pandas as pd

LP_REL = "data/1/31bonds/docplex-bin-avgonly.lp"


In [ ]:
baseline_results = {
    "LP file": LP_PATH,
    "Bonds selected": int(np.sum(x)),
    "Objective value": round(fx, 4),
    "Runtime (s)": round(dt, 2),
    "Solver": "python-mip",
    "Strategy": "docplex-bin-avgonly"
}

baseline_results


In [ ]:
import pandas as pd

results_df = pd.DataFrame([baseline_results])
results_df


### Tuned variant (20s limit)
This solver ran with a 20-second time limit, offering a faster alternative to the baseline.


## Solver Comparison Table

| Run | Solver         | LP File                   | Bonds | Objective | Runtime (s) | Notes          |
|-----|----------------|---------------------------|-------|-----------|-------------|----------------|
| 1   | Baseline (MIP) | docplex-bin-avgonly.lp    | 21    | 40.2771   | 139.88      | python-mip CLP |


In [1]:
# Step 0: Imports (if needed)
from pathlib import Path

# Step 1: Define your project root
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

# Step 2: Build relative path to your LP file
lp_rel_path = "data/1/31bonds/docplex-bin-avgonly.lp"
lp_full_path = PROJECT_ROOT / lp_rel_path

# Step 3: Check existence and read contents
if lp_full_path.exists():
    with open(lp_full_path, "r") as f:
        lp_contents = f.read()
    print("✅ File read successfully")
    print(f"📄 Preview:\n{lp_contents[:500]}")  # Optional: show first 500 chars
else:
    print(f"❌ File not found at: {lp_full_path}")


✅ File read successfully
📄 Preview:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: VCIT _ VCIT (bench)

Minimize
 obj: [ 72 fund_enriched.pmv_tgt_Investment_Grade_Credit_Capital_Goods^2
      + 72 fund_enriched.pmv_tgt_Investment_Grade_Credit_Insurance^2
      + 72 fund_enriched.pmv_tgt_Investment_Grade_Credit_Transportation^2 ]/2
Subject To
 trade_num_upper_limit: iTrade_020002BJ9 + iTrade_026874DS3 + iTrade_081437AT2
                        + iTrade_097023CJ2 + iTrade_13645RAD6 + iTrade_13645RBF0



In [ ]:
from pathlib import Path

CWD = Path.cwd()
print("CWD:", CWD)

lp_rel = Path("data/1/31bonds/docplex-bin-avgonly.lp")
lp_abs_guess = (CWD / lp_rel).resolve()
print("Expected LP path:", lp_abs_guess)
print("Exists?", lp_abs_guess.exists())


In [ ]:
from pathlib import Path

root = Path.cwd()
matches = list(root.rglob("docplex-bin-avgonly.lp"))
print("Matches found:", len(matches))
for i, p in enumerate(matches[:10], 1):
    print(f"{i}. {p}")

# Pick the first match if any were found
LP_ABS = matches[0] if matches else None
LP_ABS


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
LP_ABS = LP_ABS.resolve()
print("Found LP file at:", LP_ABS)

# Try to compute a clean relative path (if it’s inside the project)
try:
    LP_REL = str(LP_ABS.relative_to(PROJECT_ROOT))
except ValueError:
    # If it’s outside the project, just use the absolute path
    LP_REL = str(LP_ABS)

print("LP_REL set to:", LP_REL)


In [ ]:
from pathlib import Path

path_to_open = Path(LP_REL)
if not path_to_open.is_absolute():
    path_to_open = (Path.cwd() / path_to_open).resolve()

print("Opening:", path_to_open)
with open(path_to_open, "r", encoding="utf-8", errors="ignore") as f:
    head = "".join([next(f) for _ in range(5)])
print("First lines:\n", head)


In [ ]:
LP_REL = str(lp_rel)
print("LP_REL set to:", LP_REL)


In [ ]:
from pathlib import Path

path_to_open = Path(LP_REL)
if not path_to_open.is_absolute():
    path_to_open = (Path.cwd() / path_to_open).resolve()

print("Opening:", path_to_open)
with open(path_to_open, "r", encoding="utf-8", errors="ignore") as f:
    head = "".join([next(f) for _ in range(5)])
print("First lines:\n", head)


In [ ]:
# Assuming solve_mip_tuned is already defined and imported in your notebook
x2, fx2, dt2 = solve_mip_tuned(LP_REL, max_seconds=20, mip_gap=0.0)

print("🔢 Bonds selected:", int(np.sum(x2)))
print("📈 Objective value:", round(fx2, 4))
print("⏱ Runtime (seconds):", round(dt2, 2))


In [ ]:
import os
print(os.getcwd())


In [ ]:
with open("../data/1/31bonds/docplex-bin-avgonly.lp", "r") as f:
    lp_contents = f.read()
